# Phase 1: Data Ingestion and Relational Mapping
### Project: Explainable Review Intelligence System

**Objective:** The goal of this stage is to ingest large-scale unstructured review data and map it to business metadata. This creates the foundation for high-dimensional feature engineering (50+ features) by linking customer sentiment to specific business attributes.

**Data Sources:**
* `yelp_academic_dataset_review.json`: Customer ratings and text.
* `yelp_academic_dataset_business.json`: Business categories and attributes.

**Note:** If you are someone reviewing this project, or want to run it on your own device,  make sure to download the <a href='https://business.yelp.com/data/resources/open-dataset/'>yelp open dataset</a>. After that, make sure that in your root directory you have a data folder where you will place the json files in a 'raw' folder.

----

## 1. Importing Libraries

We will import the libraries here

In [1]:
import pandas as pd

----

## 2. Memory-Efficient Data Loading
To maintain system stability while handling multi-gigabyte files, we are implementing a **chunked loading strategy**. 

* **Strategy:** Sample 100,000 records from the `review` dataset.
* **Logic:** This provides a statistically significant sample size for behavior analysis while staying within RAM limits.

In [2]:
def load_rows(file_path, nrows):
    with open(file_path, 'r', encoding='utf-8') as f:
        # Empty list to store json objects in
        data = []
        
        # Index to keep track of the number of lines read 
        index = 0

        # Looping over the content of the file
        for line in f:
            if index >= nrows:
                break
            # Extracting json object from the line
            json_object = json.loads(line)
            # Appending the object to the data list
            data.append(json_object)
            # Incrementing the index
            index = index + 1

            # Returning a dataframe from the data list
        return pd.DataFrame(data)

In [3]:
reviews_df = load_rows('./data/raw/yelp_academic_dataset_review.json', 100000)
businesses_df = pd.read_json('./data/raw/yelp_academic_dataset_business.json', lines=True)

In [4]:
reviews_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
businesses_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


---

## 3. Relational Merging (Many-to-One)
We are performing an **Inner Join** between the sampled Reviews and the full Business dataset.

* **Key:** `business_id`
* **Purpose:** To enrich each review with context (Industry, Location, and Service Attributes). This allows us to move from "What is the star rating?" to "Which business attributes drive this rating?"

In [17]:
df = pd.merge(businesses_df, reviews_df, on='business_id', how='inner')
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,9vwYDBVI3ymdqcyJ5WW2Tg,e0imecnX_9MtLnS2rUZM-A,5.0,3,2,1,I've had acupuncture treatments with Abby over...,2012-05-02 18:07:38
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",-WXMS4p3D9NQsAPw4YPEyw,Jks_uMtTZHqP-84wSZ3COg,5.0,0,0,0,I have a po box there and ea. visit I am greet...,2014-09-15 14:37:42
2,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",z7TqAKXXArEB6LH6Nfr9BQ,trf3Qcz8qvCDKXiTgjUcEg,3.0,1,0,1,"Bottom Line: \nClean store, Quick Service, Go...",2011-08-01 03:45:56
3,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",8Di0vZGcRLVNCZ-AWKgshA,auE6cx-AMcv2fv4SW_gnzA,5.0,0,0,0,I went in to ship a package to my friend for h...,2018-03-06 03:17:02
4,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,"Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",IOmiYoBPtQsY_fh5uA4mXg,P-NTOAMFVSDFGkhcj4GaIQ,4.0,1,0,0,We are fans of Target. They seem to have a li...,2017-02-19 15:11:22


---

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   100000 non-null  object 
 1   name          100000 non-null  object 
 2   address       100000 non-null  object 
 3   city          100000 non-null  object 
 4   state         100000 non-null  object 
 5   postal_code   100000 non-null  object 
 6   latitude      100000 non-null  float64
 7   longitude     100000 non-null  float64
 8   stars_x       100000 non-null  float64
 9   review_count  100000 non-null  int64  
 10  is_open       100000 non-null  int64  
 11  attributes    98369 non-null   object 
 12  categories    99996 non-null   object 
 13  hours         94176 non-null   object 
 14  review_id     100000 non-null  object 
 15  user_id       100000 non-null  object 
 16  stars_y       100000 non-null  float64
 17  useful        100000 non-null  int64  
 18  funny

---

## 3. High-Dimensional Feature Extraction
The `attributes` column in the contains nested JSON/Dictionary data. To unlock deep insights, we must **flatten** these key-value pairs into individual columns.

* **Expected Outcome:** Transformation of the dataset from a narrow structure to a wide structure.
* **Process:** Extracting binary features such as 'WiFi', 'Parking', and 'NoiseLevel'.

In [19]:
# To flatten the attributes we convert each record in the column to be converted into a series
attributes_df = df['attributes'].apply(pd.Series)
df = pd.concat([df.drop('attributes', axis=1), attributes_df], axis=1)
df.sample(n=5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,AgesAllowed,DietaryRestrictions,RestaurantsCounterService
95110,WAPhi7JdwIvdTBgM9KsxHA,Westmont Diner,572 W Cuthbert Blvd,Haddon Township,NJ,08108,39.903436,-75.061846,4.0,156,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24337,Q3udDlBSVCvZSqsFtIgMLg,American Tuxedo & Bridal,3629 Gallatin Pike,Nashville,TN,37216,36.213174,-86.731752,4.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47203,vUrTGX_7HxqeoQ_6QCVz6g,Suraya,1528 Frankford Ave,Philadelphia,PA,19125,39.973687,-75.133956,4.5,1121,...,NaN,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN
49348,TV81bpCQ6p6o4Hau5hk-zw,Hellas Restaurant,785 Dodecanese Blvd,Tarpon Springs,FL,34689,28.155391,-82.761867,4.5,1368,...,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
34713,E-4t5Hoon6aVFTWDPz26fQ,Biscuits Cafe,1035 Broad Ripple Ave,Indianapolis,IN,46220,39.868795,-86.140713,4.0,308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I speculate that there could be other columns that contain JSON/Dictionary data, so let us explore the columns further

In [16]:
df.loc[:,'review_count':'AcceptsInsurance'].sample(n=10)

,review_count,is_open,categories,hours,review_id,user_id,stars_y,useful,funny,cool,...,GoodForKids,RestaurantsGoodForGroups,RestaurantsTableService,DriveThru,GoodForMeal,BusinessAcceptsBitcoin,Smoking,Music,GoodForDancing,AcceptsInsurance
7720,13,1,"Real Estate, Professional Services, Real Estat...","{'Monday': '9:30-18:0', 'Tuesday': '9:30-18:0'...",u5FJSNl9TqXQ2xIK1h7yYw,8VtQJKylp35dKgMMgbkSyg,1.0,20,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28480,157,0,"Indian, Pakistani, Restaurants","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21...",Im4peqJg2npzGgS3vFcLyQ,rQc6W9P_Pz-HcGg4BAbWPA,5.0,0,0,0,...,True,False,NaN,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN
22444,139,0,"Nightlife, Bars, Restaurants, Irish, Pubs","{'Monday': '11:0-1:0', 'Tuesday': '11:0-2:0', ...",3zH1JoRaZbfBxi_9d4DivQ,tkPb12qLYn8Ic0Un8qdb8A,3.0,1,0,0,...,False,True,NaN,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,"{'dj': False, 'background_music': False, 'no_m...",NaN,NaN
9893,323,0,"American (New), Diners, American (Traditional)...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",B8zMmUlCIX7Ts8VSgy9B-A,TGGG6aDGHxjah3_e3_695w,4.0,1,0,0,...,True,True,True,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN
61880,493,1,"American (New), Cocktail Bars, Seafood, Bars, ...","{'Monday': '0:0-0:0', 'Thursday': '15:0-22:0',...",HFpkEAiCC27O92k46cOR_Q,IIr3W4z1aU2HyNPrmEkwCw,5.0,0,0,0,...,True,True,True,NaN,"{'dessert': True, 'latenight': False, 'lunch':...",False,u'outdoor',"{'dj': False, 'background_music': False, 'juke...",True,NaN
31593,57,1,"Food, Grocery","{'Monday': '6:0-0:0', 'Tuesday': '6:0-0:0', 'W...",Tg3IbwOp_n36ESKu_JD-bQ,_7VEyrZ5gERe8sRRhqWGpw,4.0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24978,5,1,"Festivals, Arts & Entertainment",None,1B4JG92tEhsGthiZ4mi7Vg,gfQqQYI5_hCAGEHlHXIz2Q,4.0,6,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51053,65,0,"Local Flavor, Restaurants, Thai","{'Monday': '16:30-21:30', 'Tuesday': '4:30-21:...",ZIU0FAiFPyXzV5zilKPvfQ,mJhLUQUoDpgDz8cZ1To-aQ,4.0,0,0,0,...,True,True,NaN,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,NaN,NaN,NaN
26763,34,1,"Hotels & Travel, Event Planning & Services, Ho...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",YXmWgIP3wk4eqj2T-s3Ikw,0AJIpXKs5kiZLDU30_Ci4Q,4.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
70488,1014,1,"Mexican, Tex-Mex, Restaurants, Barbeque","{'Monday': '0:0-0:0', 'Tuesday': '11:0-23:0', ...",wkaDCHd76f3m7vAlmQ9vgg,sj9SXU2UrtvgqCGUECYFVg,1.0,0,0,0,...,False,True,NaN,NaN,"{'dessert': False, 'latenight': None, 'lunch':...",NaN,NaN,NaN,NaN,NaN


We can see that **BusinessParking** and **BestNights** have JSON/Dictionary data aswell. So let us flatten them and concat to our dataframe

In [13]:
business_parking_df = df['BusinessParking'].apply(pd.Series)
best_nights_df = df['BestNights'].apply(pd.Series)

df = pd.concat([df.drop(['BusinessParking', 'BestNights'], axis=1), business_parking_df, best_nights_df], axis=1)

In [14]:
df.sample(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,AgesAllowed,DietaryRestrictions,RestaurantsCounterService,0,0
26839,0cB1kDuZCvSwpJrNdtQP8Q,Giordano's,43 North Illinois St,Indianapolis,IN,46204,39.768177,-86.159639,3.5,284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': None, 'street': True, 'validated': ...",NaN
77971,aIdLyMLvkVuqUMJWuqwkbw,Jhonnys Pest Control,5878 Hollister Ave,Goleta,CA,93117,34.436084,-119.828445,4.0,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25271,MAIfD4kJDCydgoD1l2b_vw,Corks n' Crowns,32 Anacapa St,Santa Barbara,CA,93101,34.414211,-119.688757,4.0,276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': True, 'validated':...","{'monday': False, 'tuesday': False, 'friday': ..."
35783,SIoCIxjn4jLt2O-4DajWJw,Mac's Tavern,226 Market St,Philadelphia,PA,19106,39.949794,-75.144739,3.5,446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': True, 'validated':...","{'monday': False, 'tuesday': False, 'friday': ..."
80703,1m_Lh7vCDP4H8MoyO45JCw,2040 Market Street,2040 Market St,Philadelphia,PA,19103,39.953218,-75.174996,3.5,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25213,cXAKeC-EgVChIxhS7fscmw,Ghini's French Caffe,1803 E Prince Rd,Tucson,AZ,85719,32.273576,-110.944435,3.5,427,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN
28857,9Y5JPV0TFZpJXFokFxwewQ,Asian Chef Fusion Cuisine,882 Union Mill Rd,Mount Laurel,NJ,08054,39.938149,-74.915059,4.0,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN
80554,GmomRGW_omclyKXKDppsIg,Public House at Logan Square,2 Logan Sq,Philadelphia,PA,19103,39.955911,-75.169665,2.5,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': True, 'validated':...",NaN
65811,Vv5Fxre786NpKFR_3OduKw,Chimi's Fresh-Mex,2435 Woodson Rd,Saint Louis,MO,63114,38.702424,-90.362220,4.5,125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': True, 'validated':...",NaN
41413,8dEzIcKIHuOkDqLytnK_nA,Oak & More Furniture,2323 E Grant Rd,Tucson,AZ,85719,32.250723,-110.936671,4.5,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN


In [ ]:
attributes_df.loc[:,'BusinessParking':'BestNights'].sample(n=10)